In [4]:
import gspread

gc = gspread.service_account(filename="auth.json")

# create a new spreadsheet
sh = gc.create('web scraping sheet')
# share the spread with your Google accout email(the one you used for creating Google cloud project)
sh.share('nasimbd80@gmail.com', perm_type='user', role='writer')

print(sh.url)
"https://docs.google.com/spreadsheets/d/1cmtIch4iwzwxW_n08pi4BDoo37JYAILEtUYQSlhFS8A"

https://docs.google.com/spreadsheets/d/1an-uip7WPJdan1cSsMJE48yz4JUUugb8R_MTqedBPeI


'https://docs.google.com/spreadsheets/d/1cmtIch4iwzwxW_n08pi4BDoo37JYAILEtUYQSlhFS8A'

In [7]:
import gspread

gc = gspread.service_account(filename="auth.json")

# open an existing spreadsheet using differet methods
sh = gc.open("web scraping sheet")
sh = gc.open_by_key("1an-uip7WPJdan1cSsMJE48yz4JUUugb8R_MTqedBPeI")
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1an-uip7WPJdan1cSsMJE48yz4JUUugb8R_MTqedBPeI/")

print(sh)
"<Spreadsheet 'web scraping sheet' id:1an-uip7WPJdan1cSsMJE48yz4JUUugb8R_MTqedBPeI>"

<Spreadsheet 'web scraping sheet' id:1an-uip7WPJdan1cSsMJE48yz4JUUugb8R_MTqedBPeI>


"<Spreadsheet 'web scraping sheet' id:1an-uip7WPJdan1cSsMJE48yz4JUUugb8R_MTqedBPeI>"

In [12]:
import gspread
import warnings

# disable gspread migration warnings
warnings.filterwarnings("ignore")

gc = gspread.service_account(filename="auth.json")

# open the spreadsheet file and select a work sheet page(sheet1)
wk = gc.open("web scraping sheet").sheet1

# update cells by their names
wk.update("A1", "update")
wk.update("B1", "specific")
wk.update("C1", "cells")

# update cells by appending list
wk.append_row(["update", "cells by", "appending rows"])

# update a range of cells
wk.update(range_name="A3:C3", values=[["update a","range of", "cells"]])

APIError: APIError: [400]: Invalid value at 'data.values' (type.googleapis.com/google.protobuf.ListValue), "A1"

In [15]:
import gspread

gc = gspread.service_account(filename="auth.json")

# open the spreadsheet file and select a work sheet page(sheet1)
wk = gc.open("web scraping sheet").sheet1

# get all the non-empty fields as a list of lists
list_of_lists = wk.get_values()

print(list_of_lists)
"[['update', 'specific', 'cells'], ['update', 'cells by', 'appending rows'], ['update a', 'range of', 'cells']]"

[[]]


"[['update', 'specific', 'cells'], ['update', 'cells by', 'appending rows'], ['update a', 'range of', 'cells']]"

In [21]:
import gspread
import asyncio
import json
from httpx import AsyncClient, Response
from parsel import Selector
from typing import Dict, List

# 1. Create HTTP client with headers that look like a real web browser
client = AsyncClient(
    headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9,lt;q=0.8,et;q=0.7,de;q=0.6",
    }
)

def parse_data(response: Response):
    """parse product data from the HTML"""
    selector = Selector(response.text)
    data = []
    for product in selector.css(".row.product"):
        data.append({
            "name": product.css("h3 a::text").get(),
            "description": product.css(".short-description::text").get(),
            "price": product.css(".price::text").get()
        })
    return data


async def scrape_data(url: str):
    """scrape product pages"""
    response = await client.get(url)
    data = parse_data(response)
    # crawl over product pages
    for page_number in range(2, 6):
        response = await client.get(url + f"?page={page_number}")
        data.extend(parse_data(response))
    return data
    


# Main wrapper function
async def main():
    url = "https://web-scraping.dev/products"
    data = await scrape_data(url)
    print(json.dumps(data, indent=2))


# Run the async main function in an existing event loop
await main()

[
  {
    "name": "Box of Chocolate Candy",
    "description": "Indulge your sweet tooth with our Box of Chocolate Candy. Each box contains an assortment of rich, flavorful chocolates with a smooth, creamy filling. Choose from a variety of flavors including zesty orange and sweet cherry. Whether you're looking for the perfect gift or just want to treat yourself, our Box of Chocolate Candy is sure to satisfy.",
    "price": "24.99"
  },
  {
    "name": "Dark Red Energy Potion",
    "description": "Unleash the power within with our 'Dark Red Potion', an energy drink as intense as the games you play. Its deep red color and bold cherry cola flavor are as inviting as they are invigorating. Bring out the best in your gaming performance, and unlock your full potential.",
    "price": "4.99"
  },
  {
    "name": "Teal Energy Potion",
    "description": "Experience a surge of vitality with our 'Teal Potion', an exceptional energy drink designed for the gaming community. With its intriguing teal

In [31]:
import gspread
import asyncio
import json
from httpx import AsyncClient, Response
from parsel import Selector
from typing import Dict, List

# the previous code of our web-scraping.dev scraper
def scrape_data(url):
    ...

def save_to_sheet(data: List[Dict]):
    """save the result data into Google sheets"""
    gc = gspread.service_account(filename="auth.json")
    # create a new spreadsheet
    sh = gc.create('web scraping sheet')
    sh.share('nasimbd80@gmail.com', perm_type='user', role='writer')    
    # open the spreadsheet file and select a work sheet page(sheet1)
    wk = gc.open("web scraping sheet").sheet1
    # write sheet columns names
    wk.append_row(["name", "description", "price"])
    # write each product as rows:
    for product in data:
        wk.append_row([product["name"], product["description"], product["price"]])
    print(f"Data saved into the spreadsheet: ({wk.url})")


# Main wrapper function
# async def main():
#     url = "https://web-scraping.dev/products"
#     data = await scrape_data(url)
#     save_to_sheet(data)
#     "Data saved into the spreadsheet: (https://docs.google.com/spreadsheets/d/1an-uip7WPJdan1cSsMJE48yz4JUUugb8R_MTqedBPeI#gid=0)"


if __name__ == "__main__":
    data = asyncio.run(scrape_data(url="https://web-scraping.dev/products"))
    # save the data into the spreadsheet
    save_to_sheet(data)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [35]:
import gspread
import asyncio
import json
from httpx import AsyncClient, Response
from parsel import Selector
from typing import Dict, List

# HTTP client setup
client = AsyncClient(
    headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    }
)

def parse_data(response: Response):
    """Parse product data from the HTML."""
    selector = Selector(response.text)
    data = []
    for product in selector.css(".row.product"):
        data.append(
            {
                "name": product.css("h3 a::text").get(),
                "description": product.css(".short-description::text").get(),
                "price": product.css(".price::text").get(),
            }
        )
    return data

async def scrape_data(url: str):
    """Scrape product pages."""
    response = await client.get(url)
    data = parse_data(response)
    for page_number in range(2, 6):
        response = await client.get(url + f"?page={page_number}")
        data.extend(parse_data(response))
    return data

def save_to_sheet(data: List[Dict]):
    """Save the result data into Google Sheets."""
    gc = gspread.service_account(filename="auth.json")
    # Create a new spreadsheet
    sh = gc.create("web scraping data sheet")
    sh.share("nasimbd80@gmail.com", perm_type="user", role="writer")
    # Open the spreadsheet file and select a worksheet
    wk = sh.sheet1
    # Write column headers
    wk.append_row(["name", "description", "price"])
    # Write each product as rows
    for product in data:
        wk.append_row([product["name"], product["description"], product["price"]])
    print(f"Data saved into the spreadsheet: {wk.url}")

# Use await to directly call the async function in Jupyter or other environments
if __name__ == "__main__":
    url = "https://web-scraping.dev/products"
    data = await scrape_data(url)
    save_to_sheet(data)

Data saved into the spreadsheet: https://docs.google.com/spreadsheets/d/1cqoWzC7xHcV3OBW3cZ1PKsvxZtwp_k8UiVwJfqebk2I#gid=0
